In [0]:
RAW_PATH = "/Volumes/ecommerce/ecommerce/data"
GCP_PROJECT = "regal-elf-481622-u5"
BQ_DATASET = "ecommerce"
TEMP_GCS_BUCKET = "ecommerce-data1"

GCP_SECRET_SCOPE = "gcp-secrets"
GCP_SECRET_KEY = "gcp-sa-key"

# ---- Create a credentials JSON file from secret ----
key_path = "/Volumes/ecommerce/ecommerce/data/regal-elf-481622-u5-cbde293972f4.json"

spark.conf.set("credentialsFile", key_path)
spark.conf.set("parentProject", GCP_PROJECT)
spark.conf.set("temporaryGcsBucket", TEMP_GCS_BUCKET)

# Spark BigQuery connector reads credentials from this config
spark.conf.set("credentialsFile", key_path)
spark.conf.set("parentProject", GCP_PROJECT)             # billing project (good practice)
spark.conf.set("temporaryGcsBucket", TEMP_GCS_BUCKET)    # staging bucket required



In [0]:
dbutils.fs.ls("/Volumes/ecommerce/ecommerce/data/regal-elf-481622-u5-cbde293972f4.json")


In [0]:
import os, shutil, uuid

GCP_PROJECT = "regal-elf-481622-u5"
BQ_DATASET = "ecommerce"
TEMP_GCS_BUCKET = "ecommerce-data1"

# Your volume path (as you said)
volume_key_path = "/Volumes/ecommerce/ecommerce/data/regal-elf-481622-u5-cbde293972f4.json"

# Copy to a driver-local temp path
local_key_path = f"/tmp/gcp_sa_{uuid.uuid4().hex}.json"
shutil.copy(volume_key_path, local_key_path)

print("Local key exists:", os.path.exists(local_key_path), local_key_path)

# Test write
test_df = spark.createDataFrame([(1, "ok")], ["id", "status"])

(test_df.write
  .format("bigquery")
  .option("table", f"{GCP_PROJECT}:{BQ_DATASET}.dbx_test_write")
  .option("parentProject", GCP_PROJECT)
  .option("project", GCP_PROJECT)
  .option("credentialsFile", local_key_path)
  .option("temporaryGcsBucket", TEMP_GCS_BUCKET)
  .mode("overwrite")
  .save()
)

print("✅ BigQuery test write succeeded")




In [0]:
tables = {
    "daily_store_category": "ecommerce.gold.daily_store_category",
    "top_customers": "ecommerce.gold.top_customers",
    "product_impact": "ecommerce.gold.product_impact",
    "promo_impact": "ecommerce.gold.promo_impact",
    "rfm": "ecommerce.gold.rfm",
    "suspects": "ecommerce.gold.suspects",
}

for bq_table, spark_table in tables.items():
    df = spark.table(spark_table)
    target = f"{GCP_PROJECT}:{BQ_DATASET}.{bq_table}"

    print(f"Writing {spark_table} -> BigQuery {target}")

    (df.write
      .format("bigquery")
      .option("table", target)
      .option("temporaryGcsBucket", TEMP_GCS_BUCKET)
      .mode("overwrite")   # replaces table data
      .save()
    )

print("✅ BigQuery publish complete")


In [0]:
RAW_PATH = "/Volumes/ecommerce/ecommerce/data"
GCP_PROJECT = "regal-elf-481622-u5"
BQ_DATASET = "ecommerce"
TEMP_GCS_BUCKET = "ecom-databricks-temp"

GCP_SECRET_SCOPE = "gcp-secrets"
GCP_SECRET_KEY = "gcp-sa-key"

In [0]:
# Read credentials file content

spark.conf.set("temporaryGcsBucket", TEMP_GCS_BUCKET)
spark.conf.set("parentProject", GCP_PROJECT)

In [0]:
tables = {
    "daily_store_category" : "ecommerce.gold.daily_store_category",
    "top_customers" : "ecommerce.gold.top_customers",
    "product_impact" : "ecommerce.gold.product_impact",
    "promo_impact" : "ecommerce.gold.promo_impact",
    "rfm" : "ecommerce.gold.rfm",
    "suspects" : "ecommerce.gold.suspects"
}

In [0]:
spark.read.table("ecommerce.gold.suspects").display()


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print("bigQuery connector loaded")

In [0]:
for table_name, table_fqn in tables.items():
    df = spark.read.table(table_fqn)
    print(f"Writing {table_name} to BigQuery {GCP_PROJECT}.{BQ_DATASET}.{table_name}")
    df.write.mode("overwrite").option("header", "true").csv(f"dbfs:/tmp/ecommerce_exports/{table_name}.csv")